In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()



In [92]:
dfrm1=spark.read.csv('imdb_top_1000.csv',header=True,inferSchema=True)

In [93]:
dfrm1.printSchema()

root
 |-- Poster_Link: string (nullable = true)
 |-- Series_Title: string (nullable = true)
 |-- Released_Year: string (nullable = true)
 |-- Certificate: string (nullable = true)
 |-- Runtime: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- Overview: string (nullable = true)
 |-- Meta_score: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Star1: string (nullable = true)
 |-- Star2: string (nullable = true)
 |-- Star3: string (nullable = true)
 |-- Star4: string (nullable = true)
 |-- No_of_Votes: string (nullable = true)
 |-- Gross: string (nullable = true)



In [94]:
dfrm1.show()

+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|         Poster_Link|        Series_Title|Released_Year|Certificate|Runtime|               Genre|IMDB_Rating|            Overview|Meta_score|            Director|             Star1|               Star2|             Star3|               Star4|No_of_Votes|      Gross|
+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|https://m.media-a...|The Shawshank Red...|         1994|          A|142 min|               Drama|        9.3|Two imprisoned me...|        80|      Frank Darabont|       Tim Robbins|      Morgan F

In [95]:
dfrm1=dfrm1.drop('Poster_Link')

In [96]:
from pyspark.sql.functions import col,count,isnan,when
dfrm1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in dfrm1.columns]
   ).show()

+------------+-------------+-----------+-------+-----+-----------+--------+----------+--------+-----+-----+-----+-----+-----------+-----+
|Series_Title|Released_Year|Certificate|Runtime|Genre|IMDB_Rating|Overview|Meta_score|Director|Star1|Star2|Star3|Star4|No_of_Votes|Gross|
+------------+-------------+-----------+-------+-----+-----------+--------+----------+--------+-----+-----+-----+-----+-----------+-----+
|           0|            0|        101|      0|    0|          0|       0|       156|       1|    0|    0|    0|    0|          0|  169|
+------------+-------------+-----------+-------+-----+-----------+--------+----------+--------+-----+-----+-----+-----+-----------+-----+



In [97]:
from pyspark.sql.functions import mean
mean_value=dfrm1.select(mean(dfrm1['Meta_score'])).collect()[0][0]
mean_value1=dfrm1.select(mean(dfrm1['Gross'])).collect()[0][0]
dfrm1 = dfrm1.na.fill({'Certificate': 'U'})
dfrm1=dfrm1.na.fill({'Meta_score':mean_value})
dfrm1=dfrm1.na.fill({'Gross':mean_value1})
dfrm1.show()


+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------------+
|        Series_Title|Released_Year|Certificate|Runtime|               Genre|IMDB_Rating|            Overview|Meta_score|            Director|             Star1|               Star2|             Star3|               Star4|No_of_Votes|            Gross|
+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------------+
|The Shawshank Red...|         1994|          A|142 min|               Drama|        9.3|Two imprisoned me...|        80|      Frank Darabont|       Tim Robbins|      Morgan Freeman|        Bob Gunton|      William Sadler|    2343110|       

In [98]:
directors=dfrm1.filter(col('Director').isNull() | isnan(col('Director')))
directors.show()
dfrm1=dfrm1.withColumn("Director",when(col('Series_title')=="Zelig","Woody Allen").otherwise(col('Director')))


+------------+-------------+-----------+-------+------+-----------+--------------------+--------------------+--------+-----------+-----------+----------+--------------+---------------+-----+
|Series_Title|Released_Year|Certificate|Runtime| Genre|IMDB_Rating|            Overview|          Meta_score|Director|      Star1|      Star2|     Star3|         Star4|    No_of_Votes|Gross|
+------------+-------------+-----------+-------+------+-----------+--------------------+--------------------+--------+-----------+-----------+----------+--------------+---------------+-----+
|       Zelig|         1983|         PG| 79 min|Comedy|        7.7|"""Documentary"" ...| and meets variou...|    NULL|Woody Allen|Woody Allen|Mia Farrow|Patrick Horgan|John Buckwalter|39881|
+------------+-------------+-----------+-------+------+-----------+--------------------+--------------------+--------+-----------+-----------+----------+--------------+---------------+-----+



In [99]:
dfrm1.createOrReplaceTempView('dfrm1')

In [102]:
query="select * from dfrm1 where IMDB_Rating>=8.4 order by IMDB_RATING DESC"

In [103]:
q=spark.sql(query)

In [104]:
q.show()

+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------------+
|        Series_Title|Released_Year|Certificate|Runtime|               Genre|IMDB_Rating|            Overview|Meta_score|            Director|             Star1|               Star2|             Star3|               Star4|No_of_Votes|            Gross|
+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------------+
|The Shawshank Red...|         1994|          A|142 min|               Drama|        9.3|Two imprisoned me...|        80|      Frank Darabont|       Tim Robbins|      Morgan Freeman|        Bob Gunton|      William Sadler|    2343110|       